# Make Run2.1i dereddened cleaned catalogs
notebook will read in tracts one at a time, compute EBV from SFD map, add dereddened mags, and output to an HDF5 file of the appropriate format for BPZpipe<br>
Need to put in appropriate one sigma limits for objects that are not *detected*, catalog has both NaN and Inf, not sure what the difference is, let's consider both as non-detections.

NOTE: It appears that the magnitude errors for cModel mags are just defined as 1.087/SNR, which is not the form assumed by BPZ and several other photo-z codes!  Rather than grab the `magerr_{band}_cModel` quantities, instead grab the snr_{band}_cModel and calculate the "expected" form of the magnitude error as 2.5log10(1+N/S), and call this `magerrlog_band_dered`<br>

To calculate the 1 sigma limits we will grab all objects within a patch with .735<magerrlog<.77 and take the median, this will give us a rough patch-dependent estimate of the 1 sigma limit.  We can improve on this in the future

In [1]:
import sys,os
#Add the issues/330 branch that has the dr1bcatalog!
#sys.path.insert(0,'/global/homes/s/schmidt9/PZDC2/GCRreader/gcr-catalogs')

In [2]:
import GCRCatalogs
from GCR import GCRQuery
## check version
print('GCRCatalogs =', GCRCatalogs.__version__, '|' ,'GCR =', GCRCatalogs.GCR.__version__)

GCRCatalogs = 0.12.0 | GCR = 0.8.7


In [3]:
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#directory to store files
output_directory = "/global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/run2.1i_all_patches"

In [5]:
#grab the dust model
from lsst.sims.catUtils.dust import EBVbase
ebv_gen = EBVbase()

In [6]:
def compute_ebv(ra,dec):
    """
    compute ebv vector for a set of ra's and dec's, assumed to be in degrees
    inputs: ra, dec:
      vectors of sky coords in degrees
    returns:
    ebv:
      vector of E(B-V values from CCM model
    """
    ra_rad = np.radians(ra)
    dec_rad = np.radians(dec)
    eq_coords = np.array([ra_rad,dec_rad])
    ebv_vec = ebv_gen.calculateEbv(equatorialCoordinates=(eq_coords),interp=True) #Sep 3, 2019 ADD interp=True NOTE: THIS WAS NOT RE-RUN YET! ORIGINAL RUN HAD interp=False!!!
    return ebv_vec

In [7]:
def make_catalogs(gc,outdir,verbose=False):
    """
    function to make a pandas dataframe with some basic info for a tract/patch for all six bands 
    inputs:
    gc: catalog reader
    tract: int; tract number
    returns:
    nothing, but generates h5py hdf5 file with catalog and dereddened magnitudes for the tract
    """
    #outfile = f"run2.1i_tract_{tract}_dereden.hdf5"
    #outf = h5py.File(outfile,"w")
    bands = ['u','g','r','i','z','y']
    columns = ['ra','dec','extendedness','blendedness','tract','patch','objectId']
    for band in bands:
        columns.append('mag_{}_cModel'.format(band))
        columns.append('magerr_{}_cModel'.format(band))
        columns.append('snr_{}_cModel'.format(band))
        columns.append('cModelFlux_{}'.format(band))
        columns.append('cModelFluxErr_{}'.format(band))
    #simple_cuts = [GCRQuery('clean')]
    #remove the "clean" requirement and just grab everything!
    #tractfilt = f"tract=={tract}"
    #data = gc.get_quantities(columns, filters = simple_cuts, native_filters=tractfilt)
    #data = gc.get_quantities(columns, native_filters=tractfilt)
    for i,data in enumerate(gc.get_quantities(columns,return_iterator=True)):
        if i<6891:
            if i%1000 ==0:
                print(i)
            continue
        else:
            df = pd.DataFrame(data)
            ebv_vec = compute_ebv(df['ra'],df['dec'])
            df['ebv']=ebv_vec
            tract = df['tract'][0]
            patch = df['patch'][0]
            patch0,patch1 = patch.split(',')

            band_meanlam = [3671., 4827.,6223.,7546.,8691.,9710.] #just the mean for now
            band_a_ebv = np.array([4.81,3.64,2.70,2.06,1.58,1.31]) #A/E(B-V) quick calculation from CCM model!!!
            #grab the list of all available patches for the tract
            #patches = list(set(df['patch']))

            for ii,band in enumerate(bands):
                #add dereddened magnitudes and re-calculate log version of errors    
                deredden_mag = ebv_vec*band_a_ebv[ii]
                cmod_dered =df[f"mag_{band}_cModel"] - deredden_mag
                df[f"cModel_{band}_dered"]=cmod_dered
                invsn = 1./df[f"snr_{band}_cModel"]
                logmagerr = 2.5*np.log10(1.+invsn)
                df[f"magerrlog_{band}_dered"] = logmagerr

                #now, replace the non-detections in each band with 99.0 for mag and
                #the 1 sigma limit determined from 1 sigma objects in the same band
                #do this patch by patch to partially account for variable depth/conditions!
                siglow = 0.73
                sighigh = 0.77
                defaultlimmag = 25.8
                goodpatch = True
                sigselect = ((df[f'magerrlog_{band}_dered']>siglow) & (df[f'magerrlog_{band}_dered']<sighigh) \
                             & (np.isfinite(df[f'cModel_{band}_dered'])))
                if np.sum(sigselect)==0:
                    siglow = 0.71
                    sighigh = 0.79
                    sigselect = ((df[f'magerrlog_{band}_dered']>siglow) & (df[f'magerrlog_{band}_dered']<sighigh) \
                                 & (np.isfinite(df[f'cModel_{band}_dered'])))
                if np.sum(sigselect)==0:
                    print(f"bad data in patch {patch} for band {band}: no 1 sig objects, put in hard coded 25.8 limit")
                    goodpatch = False
                if verbose: print(f"{np.sum(sigselect)} total in cut for patch {patch}")
                if goodpatch:
                    sigmag = df[f'cModel_{band}_dered'][sigselect]
                    limmag = np.median(sigmag)
                    defaultlimmag = limmag
                else:
                    limmag = 25.8 #hardcoded temp solution
                if verbose: print(f"1 sigma mag for patch {patch} in band {band} is {limmag}")
                #find all NaN and Inf and replace with mag = 99 and magerr = 1 sigma limit
                nondet = ((~np.isfinite(df[f'cModel_{band}_dered']) | (~np.isfinite(df[f'magerrlog_{band}_dered']))))
                df[f'cModel_{band}_dered'][nondet] = 99.0
                df[f'magerrlog_{band}_dered'][nondet] = limmag
                if verbose: print(f"replacing inf and nan for {np.sum(nondet)} {band} band detects for patch {patch}")
            outfile = f"Run2.1i_tract_{tract}_patch_{patch0}_{patch1}_idx_{i}.hdf5"
            print (f"writing out outfile {outfile}...")
            make_output_hdf5_file(df,outfile,outdir)
    

In [8]:
def make_output_hdf5_file(df,outfile,outdir):
    """
    create h5py format hdf5 file with the format expected by BPZpipe_pzpdf
    input:
    df: pandas dataframe generated by function above
    tract: string/int: specific tract file for dataframe
    outdir: string: file path to output directory in which to save hdf5 file
    returns:
    nothing, but writes out hdf5 file to outdir
    """
    bands = ['u','g','r','i','z','y']
    fullpath = os.path.join(outdir,outfile)
    outf = h5py.File(fullpath,"w")
    group = outf.create_group("photometry")
    group['id']=df['objectId']
    group['extendedness']=df['extendedness']
    group['blendedness']=df['blendedness']
    #group['tract']=df['tract']
    #group['patch']=df['patch']
    group['ra']=df['ra']
    group['dec']=df['dec']
    for band in bands:
        group[f'mag_{band}_lsst']=df[f'cModel_{band}_dered']
        group[f'mag_err_{band}_lsst']=df[f'magerrlog_{band}_dered']
    outf.close()
    print (f"wrote out file {outfile} to {fullpath} ")
    

In [9]:
catalog_name = 'dc2_object_run2.1i_dr1'
#catalog_name = 'dc2_object_run2.1i_dr1_tract4850'

In [10]:
gc = GCRCatalogs.load_catalog(catalog_name)

In [11]:
make_catalogs(gc,output_directory,verbose=False)

0
1000
2000
3000
4000
5000
6000


/opt/lsst/software/stack/python/miniconda3-4.5.12/envs/lsst-scipipe-1172c30/lib/python3.7/site-packages/ipykernel/__main__.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/lsst/software/stack/python/miniconda3-4.5.12/envs/lsst-scipipe-1172c30/lib/python3.7/site-packages/ipykernel/__main__.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


writing out outfile Run2.1i_tract_5070_patch_1_4_idx_6891.hdf5...
wrote out file Run2.1i_tract_5070_patch_1_4_idx_6891.hdf5 to /global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/run2.1i_all_patches/Run2.1i_tract_5070_patch_1_4_idx_6891.hdf5 
writing out outfile Run2.1i_tract_5070_patch_1_5_idx_6892.hdf5...
wrote out file Run2.1i_tract_5070_patch_1_5_idx_6892.hdf5 to /global/projecta/projectdirs/lsst/groups/PZ/PhotoZDC2/run2.1i_all_patches/Run2.1i_tract_5070_patch_1_5_idx_6892.hdf5 
bad data in patch 1,6 for band u: no 1 sig objects, put in hard coded 25.8 limit
bad data in patch 1,6 for band g: no 1 sig objects, put in hard coded 25.8 limit
bad data in patch 1,6 for band r: no 1 sig objects, put in hard coded 25.8 limit
bad data in patch 1,6 for band i: no 1 sig objects, put in hard coded 25.8 limit
bad data in patch 1,6 for band z: no 1 sig objects, put in hard coded 25.8 limit
bad data in patch 1,6 for band y: no 1 sig objects, put in hard coded 25.8 limit
writing out outfile Run2